You can read the paper for more detail. <BR>
Reference: https://ieeexplore.ieee.org/abstract/document/5197422

In [4]:
class Matrix_Factorization:
    def __init__(self, alpha = 0.00001, iterations = 50, num_of_latent = 200, lam = 0.0005):
        """
            Some initializations, if neccesary

            attributes:
                        alpha: Learning Rate, default 0.01
                        num_iter: Number of Iterations to update coefficient with training data
                        num_of_latent: Number of latent factor.
                        lam: Regularization constant


            TODO: 1. Initialize all variables needed.
        """

        self.alpha = alpha
        self.iterations = iterations
        self.num_of_latent = num_of_latent
        self.lam = lam

    def fit(self, train):
        """
            Train: list of tuples with (User, Movie, Rating)
            num_user: Number of unique user.
            num_movie: Number of unique movie

            TODO: 2. Initialize num_user and num_movie
                  3. Save the training set.
                  4. Initialize P and Q matrix, with normal distribution with mean = 0.
                  Hint: Think about what P and Q represent, what they should do.Think about the shape too.


        """
        num_user = len(set([t[0] for t in train]))
        num_movie = len(set([t[1] for t in train]))

        self.train = train

        self.P = np.random.normal(0, 0.1, (num_user+1, self.num_of_latent)) #plus 1 to use index from 1
        self.Q = np.random.normal(0, 0.1, (num_movie+1, self.num_of_latent))

        rmse_lst = []

        """
            TODO: 5: Calculate the error, using P and Q matrix.
                  6: We need to check if the absolute value error is less than some constant. Store the previous Q and P for adaptive learning rate.
                      If it is less than that constant then we update P and Q matrix.
                      (When update, update the P and Q at the same time. Think about why it is important.)
                      Otherwise use the error to update the Q and P matrix.

                  7: For each entry update temp_mse, and append the Current iteration RMSE to rmse_lst.

        """

        for f in range(self.iterations):
            ### Random Shuffle. Why is this called?
            np.random.shuffle(self.train)

            temp_mse = 0

            previous_Q = self.Q.copy()
            previous_P = self.P.copy()

            Count = 0

            for tup in self.train:
                u,i,rating = tup
                error = rating - np.dot(self.P[u], self.Q[i])

                if abs(error) > 20 :
                    continue
                Count += 1
                temp_mse += error ** 2 + self.lam * (np.linalg.norm(self.P[u]) ** 2 + np.linalg.norm(self.Q[i]) ** 2)

                P = self.P[u] + self.alpha * (error * self.Q[i] - self.lam * self.P[u])
                Q = self.Q[i] + self.alpha * (error * self.P[u] - self.lam * self.Q[i])

                #### Don't Modify this code, helpful for converge.
                if np.isinf(Q).any() or np.isinf(P).any() or np.isnan(Q).any() or np.isnan(P).any():
                    pass
                else:
                    # pass
                    #### NEED TO MODIFY #### Update P and Q.
                  self.Q[i] = Q
                  self.P[u] = P

            rmse_lst.append(np.sqrt(temp_mse / Count))


            """
                TODO: 8: Implement the adaptive learning rate.
                         If current rmse is less than previous iteration, let's increase by a factor range from 1 - 1.5
                         Otherwise we decrease by a factor range from 0.5 - 1
                      9: If the current rmse is greater than previous iteration.
                         Check the relative error, (previous - current)/ previous.
                         If it is greater than 0.1, we restore the previous Q and P. (Try without it. Think about why we need this.)
            """
            if len(rmse_lst) <= 1:
              continue

            if rmse_lst[-1] < rmse_lst[-2]:
                self.alpha *= np.random.uniform(1, 1.5)
            else:
                self.alpha *= np.random.uniform(0.5, 1)
                if (rmse_lst[-2] - rmse_lst[-1]) / rmse_lst[-2] > 0.1:
                    self.Q = previous_Q
                    self.P = previous_P

        self.rmse = rmse_lst

    def ind_predict(self, tup):
        """
            tup: One single entry, (user, movie)

            TODO: 10: Use P and Q to make prediction on single entry.

        """
        u,i,_ = tup
        return np.dot(self.P[u], self.Q[i])
    def predict(self, X):
        """
            X: list of entries

            TODO: 11: Use ind_predict we create to make predicitons.
        """
        res = []
        for i in X:
            res.append(self.ind_predict(i))
        return res

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
!unzip ml-latest-small.zip

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [8]:
data_set = pd.read_csv('./ml-latest-small/ratings.csv')
data_set

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [9]:
ratings = pd.read_csv('./ml-latest-small/ratings.csv')
ratings = ratings.sort_values('timestamp').reset_index(drop = True).astype(int)
a = np.sort(ratings.movieId.unique())
index_dict = {}
for i in range(len(a)):
    index_dict[a[i]]  = i
ratings.movieId = ratings.movieId.apply(lambda x: index_dict[x])

grouped_userId = list(ratings.groupby(['userId']).apply(lambda x: x[['movieId','rating']].values))
grouped_movieId = list(ratings.groupby(['movieId']).apply(lambda x: x[['userId','rating']].values))

In [10]:
np.random.seed(0)
train = set()
test = set()

for i in range(len(grouped_movieId)):
    temp_len = len(grouped_movieId[i])
    for j in range(temp_len):
        if j < temp_len * 0.8:
            train.add((grouped_movieId[i][j][0]-1,i - 1, grouped_movieId[i][j][1]))
        else:
            test.add((grouped_movieId[i][j][0] -1, i - 1,grouped_movieId[i][j][1]))
train = list(train)
test = list(test)

#### Try to understand what is done for data cleaning.

#### If you are doing things correctly (with tuning), you will get something lower than 1.1 test RMSE.

In [11]:
%%time
clf = Matrix_Factorization()
clf.fit(train)

CPU times: user 2min 32s, sys: 659 ms, total: 2min 33s
Wall time: 2min 35s


In [12]:
error = 0
pred = clf.predict(test)
for i in range(len(test)):
    error += (test[i][2] - pred[i])**2

In [13]:
(error/ len(test))** (0.5)

1.4787755902490094

In [14]:
class Matrix_Factorization_with_bias:
    def __init__(self, alpha = 0.00001, iterations = 50, num_of_latent = 200, lam = 0.01):

        """
            Some initializations, if neccesary

            attributes:
                        alpha: Learning Rate, default 0.01
                        num_iter: Number of Iterations to update coefficient with training data
                        num_of_latent: Number of latent factor.
                        lam: Regularization constant


            TODO: 1. Initialize all variables needed.
        """

        self.alpha = alpha
        self.iterations = iterations
        self.num_of_latent = num_of_latent
        self.lam = lam



    def fit(self, train):
        """
            Train: list of tuples with (User, Movie, Rating)
            num_user: Number of unique user.
            num_movie: Number of unique movie

            TODO: 2. Initialize num_user and num_movie.
                  3. Save the training set.
                  4. Initialize bu , bi and b. b is the global mean of the rating.
                  5. Initialize P and Q matrix.
                  Hint: Think about what P and Q represent, what they should do.Think about the shape too.



        """

        num_user = len(set([t[0] for t in train]))
        num_movie = len(set([t[1] for t in train]))
        self.train = train

        self.P = np.random.normal(0, 0.1, (num_user+1, self.num_of_latent))
        self.Q = np.random.normal(0, 0.1, (num_movie+1, self.num_of_latent))
        self.bu = np.zeros(num_user+1)
        self.bi = np.zeros(num_movie+1)
        self.b = np.mean([t[2] for t in train])

        rmse_lst = []

        """
            TODO: 5: Calculate the error, using P , Q , bu , bi and b.
                  6: Update the P , Q , bu , bi and b with error you calculate.
                    (Think about why we don't need to check the absolute of error)
                  7: For each entry update temp_mse, and append the Current iteration RMSE to rmse_lst.

        """


        for f in range(self.iterations):

            np.random.shuffle(self.train)

            temp_mse = 0
            previous_Q = self.Q.copy()
            previous_P = self.P.copy()
            previous_bu = self.bu.copy()
            previous_bi = self.bi.copy()

            Count = 0
            for tup in self.train:
                u,i,rating = tup

                prediction = self.b + self.bu[u] + self.bi[i] + np.dot(self.P[u], self.Q[i])
                error = rating - prediction
                temp_mse += error ** 2 + self.lam * (np.linalg.norm(self.P[u]) ** 2 + np.linalg.norm(self.Q[i]) ** 2 + self.bu[u] ** 2 + self.bi[i] ** 2)
                Count += 1

                self.bu[u] += self.alpha * (error - self.lam * self.bu[u])
                self.bi[i] += self.alpha * (error - self.lam * self.bi[i])
                self.b += self.alpha * (error - self.lam * self.b)

                self.P[u] += self.alpha * (error * self.Q[i] - self.lam * self.P[u])
                self.Q[i] += self.alpha * (error * self.P[u] - self.lam * self.Q[i])

            rmse_lst.append(np.sqrt(temp_mse) / Count)



            """
                TODO: 8: Implement the adaptive learning rate.
                         If current rmse is less than previous iteration, let's increase by a factor range from 1 - 1.5
                         Otherwise we decrease by a factor range from 0.5 - 1
                      9: If the current rmse is greater than previous iteration.
                         Check the relative error, (previous - current)/ previous.
                         If it is greater than 0.1, we restore the previous Q and P. (Try without it. Think about why we need this.)
            """

            if len(rmse_lst) >= 2:
              if rmse_lst[-1] < rmse_lst[-2]:
                  self.alpha *= np.random.uniform(1, 1.5)
              else:
                  self.alpha *= np.random.uniform(0.5, 1)
                  if (rmse_lst[-2] - rmse_lst[-1]) / rmse_lst[-2] > 0.1:
                      self.Q = previous_Q
                      self.P = previous_P

        self.rmse = rmse_lst

    def ind_predict(self, tup):
        """
            tup: One single entry, (user, movie)

            TODO: 10: Use P and Q to make prediction on single entry.

        """
        u,i,_ = tup
        return self.b + self.bu[u] + self.bi[i] + np.dot(self.P[u], self.Q[i])


    def predict(self, X):
        """
            X: list of entries

            TODO: 11: Use ind_predict we create to make predicitons.
        """
        res = []
        for i in X:
            res.append(self.ind_predict(i))
        return res




### If you are doing thing correctly (With tuning), you should get a Test RMSE below 1.

In [15]:
%%time
clf = Matrix_Factorization_with_bias(iterations = 100, num_of_latent = 200)
clf.fit(train)

CPU times: user 4min 12s, sys: 12.2 s, total: 4min 24s
Wall time: 4min 17s


In [16]:
error = 0
pred = clf.predict(test)
for i in range(len(test)):
    error += (test[i][2] - pred[i])**2

In [17]:
(error/ len(test))** (0.5)

0.9903993100743376